## Install Packages

In [ ]:
# Setup
!pip install -U pyTigerGraph googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.6.0-py3-none-any.whl size=38554 sha256=0209301f106afecf5282275de607b4a09e1c0d95ab04287f0deed0430d09b9af
  Stored in directory: /root/.cache/pip/wheels/80/db/c0/6d958585fa97b20e250bf437acf7e6e715b4809c2dd4e55367
Successfully built googlemaps


## Add Imports and Establish Initial Connection

In [ ]:
# Imports
from datetime import datetime
from getpass import getpass
import json

from google.colab import drive
import googlemaps
import pandas as pd
import pyTigerGraph as tg
import requests

drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Connection parameters
hostName = ''
userName = ''
password = ''
conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

conn.graphname='TigyoreGraph'
secret = conn.createSecret()
authToken = conn.getToken(secret)
authToken = authToken[0]
conn = tg.TigerGraphConnection(host=hostName, graphname='TigyoreGraph', username=userName, password=password, apiToken=authToken)

print('Connected')

Connected


In [ ]:
!ls "/gdrive/My Drive/Tigyore"

ls: cannot access '/gdrive/My Drive/Tigyore': No such file or directory


In [ ]:
# API_KEY = getpass('Enter Google API Key:')

# API_KEY[:5] + '...'

In [ ]:
# https://towardsdatascience.com/geocoding-singapore-coordinates-onemap-api-3e1542bf26f7
def get_sg_lat_lng_coordinates(address):
    req = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+address+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
    resultsdict = eval(req.text)
    if len(resultsdict['results'])>0:
        return resultsdict['results'][0]['LATITUDE'], resultsdict['results'][0]['LONGITUDE']
    else:
        pass

In [ ]:
# https://github.com/googlemaps/google-maps-services-python
def get_sg_transit_directions(origin: str, destination: str, departure_time: str, api_key: str) -> dict:
    gmaps = googlemaps.Client(key=api_key)

    origin = get_sg_lat_lng_coordinates(origin)
    destination = get_sg_lat_lng_coordinates(destination)

    departure_time = datetime.strptime(departure_time, '%Y-%m-%d %H:%M:%S %z')

    directions_result = gmaps.directions(origin,
                                         destination,
                                         mode='transit',
                                         language='en',
                                         region='sg',
                                         departure_time=departure_time)
    return directions_result

In [ ]:
# https://github.com/googlemaps/google-maps-services-python
def get_nyc_transit_directions(origin: tuple, destination: tuple, departure_time: str, api_key: str) -> dict:
    gmaps = googlemaps.Client(key=api_key)

    departure_time = datetime.strptime(departure_time, '%Y-%m-%d %H:%M:%S %z')

    directions_result = gmaps.directions(origin,
                                         destination,
                                         mode='transit',
                                         language='en',
                                         region='us',
                                         departure_time=departure_time)
    return directions_result

In [ ]:
# help(googlemaps.directions)

In [ ]:
# directions_dict = get_sg_transit_directions('Tanah Merah Ferry Road', 'DUO Tower', '2022-04-16 14:00:00 +0800', API_KEY)
# directions_dict

In [ ]:
# with open('/gdrive/My Drive/Tigyore/data/sample_directions_response.json', 'w') as f:
#     f.write(json.dumps(directions_dict))

In [ ]:
directions_dict = dict()

with open('/gdrive/My Drive/Tigyore/data/sample_directions_response.json', 'r') as f:
    directions_dict = json.loads(f.readlines()[0])

In [ ]:
directions_dict

[{'bounds': {'northeast': {'lat': 1.3321295, 'lng': 103.9894148},
   'southwest': {'lat': 1.2993733, 'lng': 103.85571}},
  'copyrights': 'Map data ©2022 Google',
  'legs': [{'arrival_time': {'text': '2:59pm',
     'time_zone': 'Asia/Singapore',
     'value': 1650092358},
    'departure_time': {'text': '2:00pm',
     'time_zone': 'Asia/Singapore',
     'value': 1650088800},
    'distance': {'text': '18.1 km', 'value': 18051},
    'duration': {'text': '59 mins', 'value': 3558},
    'end_address': '3 Fraser St, Singapore 189352',
    'end_location': {'lat': 1.2994335, 'lng': 103.8581633},
    'start_address': '50 Tanah Merah Ferry Rd, Tanah Merah Ferry Terminal, Singapore 498833',
    'start_location': {'lat': 1.3142975, 'lng': 103.9882265},
    'steps': [{'distance': {'text': '0.3 km', 'value': 269},
      'duration': {'text': '3 mins', 'value': 195},
      'end_location': {'lat': 1.3151607, 'lng': 103.9888569},
      'html_instructions': 'Walk to Tanah Merah Ferry Ter',
      'polyline'

In [ ]:
arrival_time = directions_dict[0]['legs'][0]['arrival_time']['text']
departure_time = directions_dict[0]['legs'][0]['departure_time']['text']

print(departure_time, arrival_time)

2:00pm 2:59pm


In [ ]:
# Create csv of all trips - refer to notebook 1
for l in directions_dict[0]['legs'][0]['steps']:
    print(l)

{'distance': {'text': '0.3 km', 'value': 269}, 'duration': {'text': '3 mins', 'value': 195}, 'end_location': {'lat': 1.3151607, 'lng': 103.9888569}, 'html_instructions': 'Walk to Tanah Merah Ferry Ter', 'polyline': {'points': 'ku_GmeuyRKi@Qw@GQ?EMBUDC@M@?JCBAF?B@DDPPv@_@Jg@eCCGEAG@E@C@'}, 'start_location': {'lat': 1.3142975, 'lng': 103.9882265}, 'steps': [{'distance': {'text': '68 m', 'value': 68}, 'duration': {'text': '1 min', 'value': 48}, 'end_location': {'lat': 1.3145616, 'lng': 103.9888227}, 'html_instructions': 'Head <b>east</b><div style="font-size:0.9em">Restricted usage road</div>', 'polyline': {'points': 'ku_GmeuyRKi@Qw@GQ?EMB'}, 'start_location': {'lat': 1.3142975, 'lng': 103.9882265}, 'travel_mode': 'WALKING'}, {'distance': {'text': '86 m', 'value': 86}, 'duration': {'text': '1 min', 'value': 63}, 'end_location': {'lat': 1.3146568, 'lng': 103.9882264}, 'html_instructions': 'Turn <b>left</b><div style="font-size:0.9em">Restricted usage road</div>', 'maneuver': 'turn-left', '

In [ ]:
# id, trip_seq_no, mode, type, from_lat_lng, to_lat_lng, from_name, to_name, start_time, end_time
# 1, 1, WALKING, "", (1.3148164, 103.9881745), (1.3148164, 103.9881745), AAA, BBB, ...
# 1, 2, TRANSIT, "35", (1.3148164, 103.9881745), (1.3148164, 103.9881745), BBB, CCC, ...
# 1, 3, WALKING, "", (1.3148164, 103.9881745), (1.3148164, 103.9881745), CCC, DDD, ...
# 1, 4, TRANSIT, "East West Line", (1.3148164, 103.9881745), (1.3148164, 103.9881745), DDD, EEE, ...
# ...

TypeError: ignored

In [ ]:
with open('/gdrive/My Drive/Tigyore/data/sample_directions_response.json', 'r') as f:
    print(f.readlines())

['[{"bounds": {"northeast": {"lat": 1.3321295, "lng": 103.9894148}, "southwest": {"lat": 1.2993763, "lng": 103.85571}}, "copyrights": "Map data \\u00a92022 Google", "legs": [{"arrival_time": {"text": "2:59pm", "time_zone": "Asia/Singapore", "value": 1650092358}, "departure_time": {"text": "2:00pm", "time_zone": "Asia/Singapore", "value": 1650088800}, "distance": {"text": "18.1 km", "value": 18051}, "duration": {"text": "59 mins", "value": 3558}, "end_address": "3 Fraser St, Singapore 189352", "end_location": {"lat": 1.2994335, "lng": 103.8581633}, "start_address": "50 Tanah Merah Ferry Rd, Tanah Merah Ferry Terminal, Singapore 498833", "start_location": {"lat": 1.3142975, "lng": 103.9882265}, "steps": [{"distance": {"text": "0.3 km", "value": 269}, "duration": {"text": "3 mins", "value": 195}, "end_location": {"lat": 1.3151607, "lng": 103.9888569}, "html_instructions": "Walk to Tanah Merah Ferry Ter", "polyline": {"points": "ku_GmeuyRKi@Qw@GQ?EMBUDC@M@?JCBAF?B@DDPPv@_@Jg@eCCGEAG@E@C@"}